In [26]:
import pandas as pd
import ast

In [60]:
### Functions for getting data from files

## Get sections (default file name is diagnoses.csv)
def get_sections(file_name = 'diagnoses.csv'):
    person_sections = dict()
    file = open(file_name)
    content = file.read()
    lines = content.split('\n')
    file.close()
    for (index, line) in enumerate(lines):
        cols = line.split('\t')
        if index == 0 or len(cols) <= 1: #Ignore title row and empty rows
            continue
        sections = ast.literal_eval(cols[-2])
        person_sections[cols[0]] = sections
    return person_sections

In [69]:
sections = get_sections()

In [110]:
### Methods for comparing trajectories


## Get local alignment between person1(horizontal) and person2(vertical)
def get_local_alignment(p1, p2):
    #Parameters
    match = 1
    mismatch = 0
    gap_penalty = -1
    
    #Initialisation
    matrix = [[0 for i in range(len(p2) + 1)] for j in range(len(p1) + 1)]
    for i in range(len(p1) + 1):
        matrix[i][0] = i * gap_penalty
    for i in range(len(p2) + 1):
        matrix[0][i] = i * gap_penalty
        
    #Fill
    for i in range(len(p1)):
        for j in range(len(p2)):
            p1_diagnosis = p1[i][0]
            p2_diagnosis = p2[j][0]
            left = matrix[i][j+1] + gap_penalty
            right = matrix[i+1][j] + gap_penalty
            diagonal = matrix[i][j] + (match if p1_diagnosis == p2_diagnosis else mismatch)
            matrix[i+1][j+1] = max(left, right, diagonal, 0)
    
    #Find the biggest alignment value
    MAX = None
    for row in matrix:
        row_max = max(row)
        MAX = row_max if MAX == None or row_max > MAX else MAX
        
    return MAX

get_local_alignment(sections['1'], sections['2'])

5

In [94]:
### NB comparing people, there is no need for a matrix, because comparing p1 and p2 is the same as comparing p2 and p1


4

In [105]:
2 < None

TypeError: '<' not supported between instances of 'int' and 'NoneType'

KeyError: 1